In [1]:
import os, sys
os.environ['PYTHONPATH'] = '/home/qihao/CS6207'
sys.path.append('/home/qihao/CS6207')
import pickle
import random
import subprocess
import torch.cuda
import torch
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from utils.earlystopping.protocols import EarlyStopping
from utils.test_dataloder import *
import datetime
from utils.get_time import get_time
import gc
from tqdm import tqdm
from utils.warmup import *
import torch.nn.functional as F
from models.bartmodel import Bart
from transformers import get_linear_schedule_with_warmup

2024-04-05 20:03:54.251389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
a = [1,2,3,4]
for i in range(10):
    random.shuffle(a)
    print(a)

[4, 3, 1, 2]
[1, 4, 2, 3]
[4, 3, 2, 1]
[3, 4, 2, 1]
[1, 4, 3, 2]
[3, 1, 4, 2]
[4, 1, 2, 3]
[3, 2, 1, 4]
[1, 3, 4, 2]
[3, 2, 4, 1]


In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
src_keys = ['strength', 'length', 'phrase']
tgt_keys = ['bar', 'pos', 'token', 'dur', 'phrase']

binary_dir = '/home/qihao/CS6207/binary'
words_dir = '/home/qihao/CS6207/binary/words'
hparams = {
    'batch_size': 1,
    'word_data_dir': '/home/qihao/CS6207/binary/words',
    'sentence_maxlen': 512,
    'hidden_size': 768,
    'n_layers': 6,
    'n_head': 8,
    'pretrain': '',
    'lr': 5.0e-5,
    'optimizer_adam_beta1': 0.9,
    'optimizer_adam_beta2': 0.98,
    'weight_decay': 0.001,
    'patience': 5,
    'warmup': 2500,
    'lr': 5.0e-5,
    'checkpoint_dir': '',
    'drop_prob': 0.2,
    'total_epoch': 1000,
    'infer_batch_size': 1,
    'temperature': 1.2,
    'topk': 2,
    'prompt_step': 1,
    'infer_max_step': 1024,
    'output_dir': "/home/qihao/CS6207/output_melody",
}

In [5]:
def set_seed(seed=1234):  # seed setting
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    # cuDNN在使用deterministic模式时（下面两行），可能会造成性能下降（取决于model）
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

In [6]:
def load_model(checkpoint_path, device):
    model = Bart(event2word_dict=event2word_dict, 
                 word2event_dict=word2event_dict, 
                 model_pth='',
                 hidden_size=hparams['hidden_size'], 
                 num_layers=hparams['n_layers'], 
                 num_heads=hparams['n_head'], 
                 dropout=hparams['drop_prob'],).to(device)

    model.load_state_dict(torch.load(checkpoint_path, map_location=device), strict=True)
    model.eval()
    print(f"| Successfully loaded bart ckpt from {checkpoint_path}.")
    return model

In [7]:
def xe_loss(outputs, targets):
    outputs = outputs.transpose(1, 2)
    return F.cross_entropy(outputs, targets, ignore_index=0, reduction='mean')

In [11]:
# ckpt_name = 'checkpoint_20240331:181343_lr_5e-05'
# ckpt_name = 'checkpoint_20240331:191608_lr_5e-05'
# ckpt_name = 'checkpoint_20240401:105006_lr_5e-05'
# ckpt_name = 'checkpoint_20240401:101115_lr_5e-05'
# ckpt_name = 'checkpoint_20240401:162740_lr_5e-05'  ## sentence level, best performed
# ckpt_name = 'checkpoint_20240401:193940_lr_5e-05'  ## sentence level, with note number indicator
# ckpt_name = 'checkpoint_20240401:174602_lr_5e-05'  ## whole song level
# ckpt_name = 'checkpoint_20240401:202138_lr_5e-05'  ## sentence level, place duration before pos
# ckpt_name = 'checkpoint_20240402:171450_lr_5e-05'  ## sentence level, with note number indicator, with basic melody
# ckpt_name = 'checkpoint_20240403:144807_lr_5e-05'  ## whole song level, with note number indicator, with basic melody
# ckpt_name = 'checkpoint_20240405:104538_lr_5e-05'  ## whole song level, with only prosody
# ckpt_name = 'checkpoint_20240404:184614_lr_5e-05'
# ckpt_name = 'checkpoint_20240405:120733_lr_5e-05'  ## whole song level, compound word template
ckpt_name = 'checkpoint_20240405:200713_lr_5e-05'  

In [12]:
def infer_l2m():
    ## -------------------
    ##     Bart Model
    ## -------------------
    set_seed()
    print(f"Using device: {device} for inferences custom samples")
    
    # training conditions (for naming the ckpt)
    lr = hparams['lr']

    ckpt_dir = f'/home/qihao/CS6207/checkpoints/{ckpt_name}'
    ckpt_path = os.path.join(ckpt_dir, 'best.pt')

    # load dictionary
    event2word_dict, word2event_dict = pickle.load(open(f"{binary_dir}/music_dict.pkl", 'rb'))
    
    test_dataset = L2MDataset('test', event2word_dict, hparams, shuffle=False)
    test_loader = build_dataloader(dataset=test_dataset, shuffle=False, batch_size=hparams['infer_batch_size'], endless=False)
    
    print(f"Test Datalodaer = {len(test_loader)} Songs")

    # load melody generation model based on skeleton framework
    model = Bart(event2word_dict=event2word_dict, 
                 word2event_dict=word2event_dict, 
                 model_pth='',
                 hidden_size=hparams['hidden_size'], 
                 num_layers=hparams['n_layers'], 
                 num_heads=hparams['n_head'], 
                 dropout=hparams['drop_prob'],).to(device)
    model.load_state_dict(torch.load(ckpt_path, map_location=device), strict=True)
    model.eval()
    print(f"| Successfully loaded bart ckpt from {ckpt_path}.")

    # -------------------------------------------------------------------------------------------
    # Inference file path
    # -------------------------------------------------------------------------------------------
    exp_date = get_time()
    melody_output_dir = os.path.join(hparams['output_dir'], f'melody_{exp_date}')
    gt_output_dir = os.path.join(hparams['output_dir'], f'melody_gt_{exp_date}')
    if not os.path.exists(melody_output_dir):
        os.mkdir(melody_output_dir)
    if not os.path.exists(gt_output_dir):
        os.mkdir(gt_output_dir)

    # num_sample = 53
    # random_integers = random.sample(range(0, len(test_loader)-1), num_sample)
    
    for data_idx, data in enumerate(test_loader):
        # if data_idx not in random_integers:
            # continue
        try:
            data_name = data['item_name'][0] if '.mid' not in data['item_name'][0] else data['item_name'][0][:-4]

            enc_inputs = {k: data[f'src_{k}'].to(device) for k in src_keys}
            dec_inputs = {k: data[f'tgt_{k}'].to(device) for k in tgt_keys}
            
            # prompt_step = dec_inputs['token'].shape[-1]  ## output gt
            # prompt_step = hparams['prompt_step']  ## select random step
            ## select first bar
            prompt_step = 1
            bar_cnt = 0
            
            
            '''
            while bar_cnt < 2:
                # print(data['tgt_token'].squeeze().shape)
                cur_token = int(data['tgt_token'].squeeze()[prompt_step])
                cur_word = word2event_dict['Token'][cur_token]
                # print(cur_word)
                if cur_word == "Bar":
                    bar_cnt += 1
                prompt_step += 1
            '''
            
            
            dec_inputs_selected = {
                'bar': dec_inputs['bar'][:, :prompt_step],
                'pos': dec_inputs['pos'][:, :prompt_step],
                'token': dec_inputs['token'][:, :prompt_step],
                'dur': dec_inputs['dur'][:, :prompt_step],
                'phrase': dec_inputs['phrase'][:, :prompt_step],
            }
            
            _ = model.infer(enc_inputs=enc_inputs, 
                            dec_inputs_gt=dec_inputs_selected, 
                            sentence_maxlen=hparams['infer_max_step']-prompt_step, 
                            temperature=hparams['temperature'], 
                            topk=hparams['topk'], 
                            device=device, 
                            output_dir=melody_output_dir, 
                            midi_name=data_name,
                            dec_labels=dec_inputs,
                            gt_output_dir=gt_output_dir)
            
            print(f"Generating {data_idx+1}/{len(test_loader)}, Name: {data_name}")
        except Exception as e:
            traceback.print_exc()
            print(f"-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-!-\nBad Item: {data_name}")

In [13]:
infer_l2m()

Using device: cuda:0 for inferences custom samples
Test Datalodaer = 53 Songs
| Successfully loaded bart ckpt from /home/qihao/CS6207/checkpoints/checkpoint_20240405:200713_lr_5e-05/best.pt.


  1%|▎                                         | 9/1023 [00:00<00:11, 88.14it/s]

/home/qihao/CS6207/output_melody/melody_20240405:203238/没有你陪伴我真的好孤单_seg0_1_Seg1.mid
/home/qihao/CS6207/output_melody/melody_gt_20240405:203238/没有你陪伴我真的好孤单_seg0_1_Seg1.mid
Generating 1/53, Name: 没有你陪伴我真的好孤单_seg0_1_Seg1


  1%|▍                                       | 11/1023 [00:00<00:09, 102.61it/s]

| WARNING: overlapping notes at ts 964
| WARNING: overlapping notes at ts 989
| WARNING: overlapping notes at ts 998
/home/qihao/CS6207/output_melody/melody_20240405:203238/剪爱_seg0_1_Seg2.mid
/home/qihao/CS6207/output_melody/melody_gt_20240405:203238/剪爱_seg0_1_Seg2.mid
Generating 2/53, Name: 剪爱_seg0_1_Seg2


...

| WARNING: overlapping notes at ts 690
| WARNING: overlapping notes at ts 732
| WARNING: overlapping notes at ts 801
/home/qihao/CS6207/output_melody/melody_20240405:203238/好不容易_seg0_1_Seg1.mid
/home/qihao/CS6207/output_melody/melody_gt_20240405:203238/好不容易_seg0_1_Seg1.mid
Generating 51/53, Name: 好不容易_seg0_1_Seg1


  1%|▍                                        | 10/1023 [00:00<00:10, 98.34it/s]

/home/qihao/CS6207/output_melody/melody_20240405:203238/独家记忆_seg0_1_Seg1.mid
/home/qihao/CS6207/output_melody/melody_gt_20240405:203238/独家记忆_seg0_1_Seg1.mid
Generating 52/53, Name: 独家记忆_seg0_1_Seg1


 43%|█████████████████▏                      | 441/1023 [00:04<00:06, 92.02it/s]


/home/qihao/CS6207/output_melody/melody_20240405:203238/酒醉的探戈2001_seg0_1_Seg2.mid
/home/qihao/CS6207/output_melody/melody_gt_20240405:203238/酒醉的探戈2001_seg0_1_Seg2.mid
Generating 53/53, Name: 酒醉的探戈2001_seg0_1_Seg2
